# README

Simulated sequecnes were generated using generated_apples_and_oranges.ipynb in this repo

lastz and minimap2 alignmnets between simulated sequecnes were generated using Galaxy and can be found in this history: https://usegalaxy.org/u/cartman/h/simulated-runs

The resulting files `minimap2.paf` and `lastz.general` are also found in this repo

Fastga alignments were generating by cloning `170a178d` version of https://github.com/thegenemyers/FASTGA and running the following command:

```
$ FastGA -T8 -i.6 orange.indels.fa apple.indels.fa > fastga.paf
```

In [1]:
import pandas as pd
import altair as alt

/tmp/ipykernel_551031/2443617991.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
%cd ~/sandbox/align_sim/

/home/anton/sandbox/align_sim


In [4]:
#Parse sequence names for APPLES
!grep '^>' apple.indels.fa | awk '{match($0, /id[0-9]+/); id=substr($0, RSTART+2, RLENGTH-2); match($0, /len[0-9]+/); len=substr($0, RSTART+3, RLENGTH-3); print substr($0,2) "\t" id "\t" len}' > apple.names.tsv

In [5]:
apple = pd.read_csv('apple.names.tsv',sep="\t",names=['seq','divergence','length'])

In [6]:
apple['labels'] = apple['seq'].str[6:]

In [7]:
apple.head()

,seq,divergence,length,labels
0,APPLE_id100_len1000,100,1000,id100_len1000
1,APPLE_id100_len2000,100,2000,id100_len2000
2,APPLE_id100_len5000,100,5000,id100_len5000
3,APPLE_id100_len10000,100,10000,id100_len10000
4,APPLE_id99_len1000,99,1000,id99_len1000


In [8]:
# Used for ordering labels in the graph
sorted_apple = list(apple.sort_values(by=['divergence','length'],ascending=True)['labels'])

## Prep lastz output

In [9]:
# Remove "%" from columns of lastz output
!awk 'BEGIN{FS=OFS="\t"} {gsub(/%/, "", $13); gsub(/%/, "", $15); print}' lastz.general > lz.tsv

In [10]:
lz = pd.read_csv('lz.tsv',sep='\t',names="score,name1,strand1,size1,zstart1,end1,name2,strand2,size2,zstart2,end2,identity_rat,identity,coverage_rat,cvrg".split(","))

In [11]:
lz.head()

,score,name1,strand1,size1,zstart1,end1,name2,strand2,size2,zstart2,end2,identity_rat,identity,coverage_rat,cvrg
0,93266,APPLE_id100_len1000,+,3000,1000,2003,ORANGE_id100_len1000,-,3011,1000,2014,1002/1003,99.9,1003/3000,33.4
1,184118,APPLE_id100_len2000,+,4000,1000,3000,ORANGE_id100_len2000,+,4029,1000,3029,1999/2000,100.0,2000/4000,50.0
2,459054,APPLE_id100_len5000,+,7000,1000,6029,ORANGE_id100_len5000,+,7080,1000,6110,5017/5029,99.8,5029/7000,71.8
3,918611,APPLE_id100_len10000,+,12000,1000,11001,ORANGE_id100_len10000,+,12176,1000,11177,9997/10001,100.0,10001/12000,83.3
4,89388,APPLE_id99_len1000,+,3000,1000,2007,ORANGE_id99_len1000,-,3019,1000,2027,995/1007,98.8,1007/3000,33.6


In [12]:
# Retain only target info
lz = apple.merge(lz, left_on='seq',right_on='name1',how='left')#[['seq','divergence','length','identity','cvrg']]

In [13]:
# Needed to correct for the lengths of unalignable flanks
lz['corrected_cvrg']=(lz['end1']-lz['zstart1'])/lz['length']

In [14]:
lz = lz[['seq','divergence','length','identity','corrected_cvrg']]

In [15]:
lz['aligner']='lastz'

In [16]:
lz.head()

,seq,divergence,length,identity,corrected_cvrg,aligner
0,APPLE_id100_len1000,100,1000,99.9,1.0030,lastz
1,APPLE_id100_len2000,100,2000,100.0,1.0000,lastz
2,APPLE_id100_len5000,100,5000,99.8,1.0058,lastz
3,APPLE_id100_len10000,100,10000,100.0,1.0001,lastz
4,APPLE_id99_len1000,99,1000,98.8,1.0070,lastz


## Process minimap2 output

In [17]:
!cut -f 1,6,7,10,11 minimap2.paf > mm.tsv

In [18]:
mm = pd.read_csv('mm.tsv',sep='\t',names='q,t,t_len,matches,align_len'.split(','))

In [19]:
mm = apple.merge(mm, left_on='seq',right_on='t',how='left')#[['seq','divergence','length','identity','cvrg']]

In [20]:
mm.head()

,seq,divergence,length,labels,q,t,t_len,matches,align_len
0,APPLE_id100_len1000,100,1000,id100_len1000,ORANGE_id100_len1000,APPLE_id100_len1000,3000.0,964.0,1004.0
1,APPLE_id100_len2000,100,2000,id100_len2000,ORANGE_id100_len2000,APPLE_id100_len2000,4000.0,1921.0,2021.0
2,APPLE_id100_len5000,100,5000,id100_len5000,ORANGE_id100_len5000,APPLE_id100_len5000,7000.0,4805.0,5076.0
3,APPLE_id100_len10000,100,10000,id100_len10000,ORANGE_id100_len10000,APPLE_id100_len10000,12000.0,9616.0,10171.0
4,APPLE_id99_len1000,99,1000,id99_len1000,ORANGE_id99_len1000,APPLE_id99_len1000,3000.0,803.0,1016.0


In [21]:
mm['identity']=mm['matches']/mm['align_len']
mm['corrected_cvrg']=mm['align_len']/mm['length']

In [22]:
mm = mm[['seq','divergence','length','identity','corrected_cvrg']]

In [23]:
mm['aligner']='minimap2'

In [24]:
#mm['identity']=mm['identity']*100
#mm['cvrg']=mm['cvrg']*100

In [25]:
mm.head()

,seq,divergence,length,identity,corrected_cvrg,aligner
0,APPLE_id100_len1000,100,1000,0.960159,1.0040,minimap2
1,APPLE_id100_len2000,100,2000,0.950520,1.0105,minimap2
2,APPLE_id100_len5000,100,5000,0.946612,1.0152,minimap2
3,APPLE_id100_len10000,100,10000,0.945433,1.0171,minimap2
4,APPLE_id99_len1000,99,1000,0.790354,1.0160,minimap2


## Process FastGa

In [26]:
!cut -f 1,6,7,10,11 fastga.paf > fga.tsv

In [27]:
fga = pd.read_csv('fga.tsv',sep='\t',names='q,t,t_len,matches,align_len'.split(','))

In [28]:
fga = apple.merge(fga, left_on='seq',right_on='t',how='left')#[['seq','divergence','length','identity','cvrg']]

In [29]:
fga.head()

,seq,divergence,length,labels,q,t,t_len,matches,align_len
0,APPLE_id100_len1000,100,1000,id100_len1000,ORANGE_id100_len1000,APPLE_id100_len1000,3000.0,1000.0,1005.0
1,APPLE_id100_len2000,100,2000,id100_len2000,ORANGE_id100_len2000,APPLE_id100_len2000,4000.0,2000.0,2014.0
2,APPLE_id100_len5000,100,5000,id100_len5000,ORANGE_id100_len5000,APPLE_id100_len5000,7000.0,5000.0,5040.0
3,APPLE_id100_len10000,100,10000,id100_len10000,ORANGE_id100_len10000,APPLE_id100_len10000,12000.0,10001.0,10089.0
4,APPLE_id99_len1000,99,1000,id99_len1000,ORANGE_id99_len1000,APPLE_id99_len1000,3000.0,995.0,1010.0


In [30]:
fga['identity']=fga['matches']/fga['align_len']
fga['corrected_cvrg']=fga['align_len']/fga['length']

In [31]:
fga = fga[['seq','divergence','length','identity','corrected_cvrg']]

In [32]:
fga['aligner']='fastga'

In [33]:
fga.head()

,seq,divergence,length,identity,corrected_cvrg,aligner
0,APPLE_id100_len1000,100,1000,0.995025,1.0050,fastga
1,APPLE_id100_len2000,100,2000,0.993049,1.0070,fastga
2,APPLE_id100_len5000,100,5000,0.992063,1.0080,fastga
3,APPLE_id100_len10000,100,10000,0.991278,1.0089,fastga
4,APPLE_id99_len1000,99,1000,0.985149,1.0100,fastga


## Plotting


In [34]:
data = pd.concat([lz,mm,fga], ignore_index=True)

In [35]:
data.head()

,seq,divergence,length,identity,corrected_cvrg,aligner
0,APPLE_id100_len1000,100,1000,99.9,1.0030,lastz
1,APPLE_id100_len2000,100,2000,100.0,1.0000,lastz
2,APPLE_id100_len5000,100,5000,99.8,1.0058,lastz
3,APPLE_id100_len10000,100,10000,100.0,1.0001,lastz
4,APPLE_id99_len1000,99,1000,98.8,1.0070,lastz


In [36]:
data['labels'] = data['seq'].str[6:]

In [37]:
data.head()

,seq,divergence,length,identity,corrected_cvrg,aligner,labels
0,APPLE_id100_len1000,100,1000,99.9,1.0030,lastz,id100_len1000
1,APPLE_id100_len2000,100,2000,100.0,1.0000,lastz,id100_len2000
2,APPLE_id100_len5000,100,5000,99.8,1.0058,lastz,id100_len5000
3,APPLE_id100_len10000,100,10000,100.0,1.0001,lastz,id100_len10000
4,APPLE_id99_len1000,99,1000,98.8,1.0070,lastz,id99_len1000


In [44]:
data.loc[(data['divergence'] >= 60) & (data['divergence'] <  70) , 'div_group']  = '60 - 69'
data.loc[(data['divergence'] >= 70) & (data['divergence'] <  80) , 'div_group']  = '70 - 79'
data.loc[(data['divergence'] >= 80) & (data['divergence'] <  90) , 'div_group']  = '80 - 89'
data.loc[(data['divergence'] >= 90) , 'div_group'] = '90 - 100'

In [45]:
data.head()

,seq,divergence,length,identity,corrected_cvrg,aligner,labels,div_group
0,APPLE_id100_len1000,100,1000,99.9,1.0030,lastz,id100_len1000,90 - 100
1,APPLE_id100_len2000,100,2000,100.0,1.0000,lastz,id100_len2000,90 - 100
2,APPLE_id100_len5000,100,5000,99.8,1.0058,lastz,id100_len5000,90 - 100
3,APPLE_id100_len10000,100,10000,100.0,1.0001,lastz,id100_len10000,90 - 100
4,APPLE_id99_len1000,99,1000,98.8,1.0070,lastz,id99_len1000,90 - 100


In [48]:
alt.Chart(data).mark_point().encode(
    x=alt.X('labels:O', sort=sorted_apple),
    y=alt.Y('corrected_cvrg:Q').axis(title=None),
    color='aligner:N',
    row='aligner:N'
).properties(
    width=1200,  # Set width of the chart
    height=50,  # Set height of the chart,
)


alt.Chart(...)

In [78]:
alt.Chart(data).mark_circle(size=100).encode(
    x=alt.X('divergence:O').axis(title=None),#, sort=sorted_apple),
    y=alt.Y('corrected_cvrg:Q').axis(title=None),
    color=alt.Color('aligner:N',sort=['lastz','fastga','minimap2']),
    row=alt.Row('length:N',title=None),
    #column='divergence:O'
).properties(
    width=800,  # Set width of the chart
    height=50,  # Set height of the chart,
).resolve_scale(
    x='independent'  # Make x-axis independent across columns
)


alt.Chart(...)